This notebook connects to Angel Broking via it's free API with an account and pulls 15 minutes data for the nifty 200. Then saving and exiting session. 

All files used in this notebook are also uploaded in the same folder. 

If you do not understand pieces of code in other files, please look here as i have explained every step.

## Connect To Angel Broking via API call

In [1]:
# package import statement
from smartapi import SmartConnect #or from smartapi.smartConnect import SmartConnect
#import smartapi.smartExceptions(for smartExceptions)

#create object of call
obj = SmartConnect(api_key="ef3Q6vdl")
                #optional
                #access_token = "your access token",
                #refresh_token = "your refresh_token")

#login api call

data = obj.generateSession(##username,##password) 
refreshToken = data['data']['refreshToken']

#fetch the feedtoken
feedToken = obj.getfeedToken()

#fetch User Profile
userProfile = obj.getProfile(refreshToken)

### Every stock has an associated token, reading the token here

In [3]:
import pandas as pd

all_symbols = pd.read_json("OpenAPIScripMaster.json")
all_symbols.head()

,token,symbol,name,expiry,strike,lotsize,instrumenttype,exch_seg,tick_size
0,22197,737JH31-SG,737JH31,,-1,100,,NSE,1
1,2372,63PY26-SG,63PY26,,-1,100,,NSE,1
2,9698,UTDGR31P12-MF,UTDGR31P12,,-1,1,,NSE,1
3,6022,ABSLAMC-BL,ABSLAMC,,-1,1,,NSE,5
4,20054,IIFCL-N2,IIFCL,,-1,1,,NSE,1


### Getting a list of the nifty 200, this can be any list of stocks for which you want data

In [4]:
nifty200 = pd.read_csv('ind_nifty200list.csv')
train_symbols = list(nifty200['Symbol'] + '-EQ')

len(train_symbols)

200

### Associating names with tokens

In [5]:
nse_stocks = []
for i in range(len(train_symbols)):
    nse_stocks.append(all_symbols[(all_symbols.exch_seg == 'NSE') & (all_symbols['symbol'] == train_symbols[i])])

token = {}
for i in nse_stocks:
    try:
        t = (i['token'].values)[0]
        sym = (i['symbol'].values)[0]
        token[sym] = (t)
    except:
        pass

In [7]:
print(token)

{'ACC-EQ': '22', 'AUBANK-EQ': '21238', 'AARTIIND-EQ': '7', 'ADANIENT-EQ': '25', 'ADANIGREEN-EQ': '3563', 'ADANIPORTS-EQ': '15083', 'ABCAPITAL-EQ': '21614', 'ABFRL-EQ': '30108', 'AJANTPHARM-EQ': '8124', 'APLLTD-EQ': '25328', 'ALKEM-EQ': '11703', 'AMARAJABAT-EQ': '100', 'AMBUJACEM-EQ': '1270', 'APOLLOHOSP-EQ': '157', 'APOLLOTYRE-EQ': '163', 'ASHOKLEY-EQ': '212', 'ASIANPAINT-EQ': '236', 'ASTRAL-EQ': '14418', 'AUROPHARMA-EQ': '275', 'DMART-EQ': '19913', 'AXISBANK-EQ': '5900', 'BAJAJ-AUTO-EQ': '16669', 'BAJFINANCE-EQ': '317', 'BAJAJFINSV-EQ': '16675', 'BAJAJHLDNG-EQ': '305', 'BALKRISIND-EQ': '335', 'BANDHANBNK-EQ': '2263', 'BANKBARODA-EQ': '4668', 'BANKINDIA-EQ': '4745', 'BATAINDIA-EQ': '371', 'BERGEPAINT-EQ': '404', 'BEL-EQ': '383', 'BHARATFORG-EQ': '422', 'BHEL-EQ': '438', 'BPCL-EQ': '526', 'BHARTIARTL-EQ': '10604', 'BIOCON-EQ': '11373', 'BOSCHLTD-EQ': '2181', 'BRITANNIA-EQ': '547', 'CADILAHC-EQ': '7929', 'CANBK-EQ': '10794', 'CASTROLIND-EQ': '1250', 'CHOLAFIN-EQ': '685', 'CIPLA-EQ': '694

### How far back do you want data for, look at API docs for details

In [8]:
from datetime import date, timedelta

today = str(date.today()- timedelta(days = 1))
today += " 09:00"
prev_15 = str(date.today() - timedelta(days = 200)) + ' 09:00'
# prev_D = str(date.today() - timedelta(days = 120)) + ' 09:00'

today

'2022-07-12 09:00'

## Pulling 15 min data

In [10]:
import time

all_data_15 = []
all_t = []
for t in token.values():    
    #Historic api
    #time.sleep(.1)
    try:
        historicParam={
            "exchange": "NSE",
            "symboltoken": str(t),
            "interval": "FIFTEEN_MINUTE",
            "fromdate": prev_15,
            "todate": today
        }
        hist = obj.getCandleData(historicParam)
        ## to make sure all dfs are same length
        # print(len(hist['data']))
        if(len(hist['data']) > 0):

            all_t.append(t)
            all_data_15.append(hist)
        else:
            print("Historic Api failed: {}".format(t))
    except Exception as e:
        print("Historic Api failed: {}".format(e))


Historic Api failed: 21238
Historic Api failed: 1633
Historic Api failed: 2643
Historic Api failed: 2664
Historic Api failed: 2535
Historic Api failed: 1442
Historic Api failed: 18921


## Converting into readable format

In [12]:
import numpy as np

for i in range(len(all_data_15)):
    try:
        all_data_15[i] = pd.DataFrame(all_data_15[i]['data'])
    except:
        pass

    real_names = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

    all_data_15[i].columns = real_names

    all_data_15[i]['timestamp'] = pd.to_datetime(all_data_15[i]['timestamp'])
    vals = list(token.values()).index(all_t[i])
    all_data_15[i]['stock'] = list(token.keys())[vals]

In [13]:
all_data_15[0].head()

,timestamp,open,high,low,close,volume,stock
0,2022-01-13 09:15:00+05:30,2323.00,2325.45,2304.0,2306.15,25250,ACC-EQ
1,2022-01-13 09:30:00+05:30,2306.15,2316.55,2305.7,2310.30,10827,ACC-EQ
2,2022-01-13 09:45:00+05:30,2310.00,2314.75,2309.0,2310.75,6726,ACC-EQ
3,2022-01-13 10:00:00+05:30,2310.00,2311.55,2298.5,2301.00,13934,ACC-EQ
4,2022-01-13 10:15:00+05:30,2300.45,2307.40,2298.2,2304.05,6619,ACC-EQ


## Scaling the Data (Optional)

In [19]:
from sklearn.preprocessing import StandardScaler
scaled_dat = []

for i in range(len(all_data_15)):
    try:
        del all_data_15[i]['timestamp']    
    except:
        pass
    scaler = StandardScaler()
    
    all_data_15[i].iloc[:,:-2] = pd.DataFrame(scaler.fit_transform(all_data_15[i].iloc[:,:-2]))
    scaled_dat.append(scaler)

In [20]:
all_data_15[i]

,open,high,low,close,volume,stock,pivot
0,2.437291,2.476657,2.461127,2.494158,-0.019218,ZEEL-EQ,-1
1,2.489343,2.495745,2.518508,2.509793,-0.272269,ZEEL-EQ,1
2,2.506694,2.500951,2.481993,2.455941,-0.250040,ZEEL-EQ,0
3,2.444232,2.421130,2.382880,2.414249,0.087342,ZEEL-EQ,0
4,2.406060,2.407248,2.429828,2.433358,-0.322732,ZEEL-EQ,0
...,...,...,...,...,...,...,...
3045,-1.071006,-1.082305,-1.051298,-1.079159,-0.434961,ZEEL-EQ,0
3046,-1.069271,-1.087511,-1.080858,-1.093056,-0.405430,ZEEL-EQ,0
3047,-1.091827,-1.101393,-1.079119,-1.091319,-0.378625,ZEEL-EQ,0
3048,-1.090092,-1.089246,-1.073903,-1.101742,-0.211742,ZEEL-EQ,0


## Save using pickle

In [21]:
import pickle

with open("test", "wb") as fp:
    pickle.dump(all_data_15, fp)

In [ ]:
import pickle

with open("scales", "wb") as fp:
    pickle.dump(scaled_dat, fp)

## Log Out

In [37]:
#logout
try:
    logout=obj.terminateSession('Your Client Id')
    print("Logout Successfull")
except Exception as e:
    print("Logout failed: {}".format(e.message))